In [1]:
from requests import get
from requests.exceptions import RequestException
from contextlib import closing
from bs4 import BeautifulSoup

In [2]:
def simple_get(url):
    """
    Attempts to get the content at `url` by making an HTTP GET request.
    If the content-type of response is some kind of HTML/XML, return the
    text content, otherwise return None
    """
    try:
        with closing(get(url, stream=True)) as resp:
            if is_good_response(resp):
                return resp.content
            else:
                return None

    except RequestException as e:
        log_error('Error during requests to {0} : {1}'.format(url, str(e)))
        return None


def is_good_response(resp):
    """
    Returns true if the response seems to be HTML, false otherwise
    """
    content_type = resp.headers['Content-Type'].lower()
    return (resp.status_code == 200 
            and content_type is not None 
            and content_type.find('html') > -1)


def log_error(e):
    """
    It is always a good idea to log errors. 
    This function just prints them, but you can
    make it do anything.
    """
    print(e)

In [4]:
raw_html = simple_get('https://www.wetter.de/')
print(raw_html)

b'<!DOCTYPE html>\n\n<!--[if lt IE 7]>\n<html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="de"> <![endif]-->\n<!--[if IE 7]>\n<html class="no-js lt-ie9 lt-ie8" lang="de"> <![endif]-->\n<!--[if IE 8]>\n<html class="no-js lt-ie9" lang="de"> <![endif]-->\n<!--[if gt IE 8]><!-->\n<html lang="de"> <!--<![endif]-->\n\n\n<head>\n    <!-- partials/site/html/head -->\n<script src="//cdn.optimizely.com/js/5078271027.js"></script>\n\n<!-- head title -->\n<title>Wetter - Wettervorhersage - Wetterbericht - wetter.de</title>\n\n<!-- head meta -->\n<meta name="msvalidate.01" content="6593CA878189EEEF3510F38B05DE90A1" >\n<meta name="description" content="Wetter &amp;#10003; Regenradar &amp;#10003; Unwetterwarnung &amp;#10003; Pollenflug &amp;#10003; Pegelst\xc3\xa4nde &amp;#10003; Skiwetter &amp;#10003; Wettervorhersage f\xc3\xbcr Deutschland &amp;#10003; - wetter.de" >\n<meta name="keywords" content="Wetter, Wettervorhersage, wetter.de, unwetter, rtl wetter, wetter aktuell, wetter online, klima, wetterbe